<a href="https://colab.research.google.com/github/hws2002/MachineLearning_PytorchNScikitLearn/blob/master/chapter15/Chapter15_3_1_RNN_with_pytorch_project1_torchtext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch==2.0.1

In [ ]:
!pip install torchtext==0.15.2
!pip install portalocker==2.1.0

In [3]:
import torch
import torch.nn as nn

### 영화 리뷰 데이터 준비
8장에서 리뷰 데이터셋을 전처리하고 정제했던 것 처럼, 여기에서도 동일한 작업을 수행해보자.  

In [4]:
from torchtext.datasets import IMDB
from torch.utils.data.dataset import random_split

In [8]:
train_dataset = IMDB(split = 'train')
test_dataset = IMDB(split = 'test')

In [6]:
print(train_dataset)

ShardingFilterIterDataPipe


In [ ]:
# ShardingFilterIterDataPipe는 Pytorch DataPipes의 한 종류임( often used for handling large datasets or distributed training)
# Since `ShardingFilterIterDataPipe` works with streaming data, it won't support direct indexing like lists
# 그래서 이런식으로 enumerate해서 예시 파일들을 볼 수 있음
for i, sample in enumerate(train_dataset):
  if i >= 5:
    break
  print(sample)

In [ ]:
# 나중에 colab 노트북을 열었을때 다시 사용할수 있도록 저장해두자
import pickle

with open('train_dataset.pkl', 'wb') as f:
  pickle.dump(train_dataset,f)

with open('test_dataset.pkl', 'wb') as f:
  pickle.dump(test_dataset,f)

# how to load
with open('train_dataset.pkl', 'rb') as f:
  loaded_train_dataset = pickle.load(f)

for i, sample in enumerate(loaded_train_dataset):
  if i>= 5:
    break
  print(sample)

## 전처리
1. 훈련 데이터셋을 훈련, 검증 데이터셋으로 나누기
2. 훈련 데이터셋에 있는 고유한 단어 찾기
3. 고유한 단어를 고유한 정수로 매핑하고 리뷰 텍스트를 정수(고유 단어의 인덱스) 배열로 인코딩하기
4. 모델에 입력하기 위해 데이터셋을 미니 배치로 나누기

각각  
1. random_split
2. tokenizer + Counter
3. torchtext의 vocab
4. dataloader  

를 사용해서 해볼것이다

In [10]:
# 1단계 : 훈련, 검증 데이터셋으로 나누기
from torch.utils.data.dataset import random_split
print(len(list(train_dataset)))
torch.manual_seed(1)
train_dataset, valid_dataset = random_split(
    list(train_dataset), [20000, 5000]
)

25000


In [11]:
# 2단계 : 고유 토큰 (단어) 찾기
# 8장에서 만든 tokenizer 함수를 사용해보자
import re
from collections import Counter, OrderedDict

token_counts = Counter()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = text.split()
    return tokenized

for label, line in train_dataset:
  tokens = tokenizer(line)
  token_counts.update(tokens)

print('vocab size : ', len(token_counts))

vocab size :  69023


In [34]:
# 3단계 : 고유 토큰을 정수로 인코딩하기
from torchtext.vocab import vocab

print(type(token_counts))
sorted_by_freq_tuples = sorted(
    token_counts.items(), key = lambda x : x[1], reverse = True
) # 빈도수를 기준으로 내림차순으로 정렬
print(type(sorted_by_freq_tuples))

ordered_dict = OrderedDict(sorted_by_freq_tuples) # 파이썬 3.6이상 버전부터는 없어도 되는 코드
vocab = vocab(ordered_dict)
print(sorted_by_freq_tuples[:10])
print(vocab.get_itos())

# 추가적으로 token_counts에 포함되지 않은 모든 토큰은 정수 1을 할당
vocab.insert_token("<unk>", 1)
# 시퀀스 길이를 조절하기 위한 패딩 토큰은 정수 0을 할당
vocab.insert_token("<pad>", 0)
vocab.set_default_index(vocab["<unk>"])
print(len(vocab))

<class 'collections.Counter'>
<class 'list'>
[('the', 267877), ('and', 130797), ('a', 130057), ('of', 116119), ('to', 107513), ('is', 85847), ('it', 76964), ('in', 74646), ('i', 69938), ('this', 60714)]
['the', 'and', 'a', 'of', 'to', 'is', 'it', 'in', 'i', 'this', 'that', 's', 'was', 'as', 'for', 'with', 'movie', 'but', 'film', 't', 'on', 'you', 'not', 'he', 'are', 'his', 'have', 'one', 'be', 'all', 'at', 'they', 'by', 'an', 'who', 'so', 'from', 'like', 'there', 'her', 'or', 'just', 'about', 'out', 'has', 'if', 'what', 'some', 'good', 'can', 'she', 'when', 'very', 'more', 'up', 'even', 'time', 'no', 'my', 'would', 'which', 'story', 'only', 'really', 'see', 'had', 'their', 'we', 'were', 'me', 'well', 'than', 'much', 'get', 'been', 'will', 'bad', 'other', 'people', 'do', 'also', 'first', 'because', 'great', 'into', 'him', 'how', 'don', 'most', 'made', 'its', 'then', 'way', 'make', 'could', 'them', 'too', 'after', 'any', 'movies', 'think', 'characters', 'watch', 'films', 'two', 'characte

In [35]:
# vocab 객체는 다음과 같이 사용할 수 있음
example_line = ['this', 'is', 'an', 'example']
print([vocab[token] for token in example_line])

[11, 7, 35, 457]


In [36]:
# 3단계 - A : 변환 함수 정의 (pipeline)
import torchtext
text_pipeline = \
  lambda x : [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x : 1. if x == 'pos' else 0.

# 3단계 - B : 텍스트 인코딩과 레이블 변환 함수를 collate_batch 함수에 감싼 후 collate_fn 매개변수에 전달하여 만든
# DataLoader로 샘플의 배치를 생성하자.
def collate_batch(batch):
  label_list, text_list, lengths = [], [], []
  for _label, _text in batch:
    label_list.append(label_pipeline(_label))
    processed_text = torch.tensor(text_pipeline(_text),
                                  dtype = torch.int64)
    text_list.append(processed_text)
    lengths.append(processed_text.size(0)) # for a 1D tensor, just the length of the tensor
  label_list = torch.tensor(label_list)
  lengths = torch.tensor(lengths)
  padded_text_list = nn.utils.rnn.pad_sequence(
      text_list, batch_first = True # the output tensor will have the shape (batch_size, sequence_length)
  )
  return padded_text_list, label_list, lengths

# 4단계
## 배치 만들기
from torch.utils.data import DataLoader
dataloader = DataLoader(train_dataset, batch_size = 4,
                        shuffle = False, collate_fn = collate_batch)

In [37]:
# 패딩 작동 방식을 이해하기 위해
# 첫 번째 배치를 선택하여 미니 배치에 들어가기 전 개별 원소의 크기와 미니 배치의 차원을 출력해보자
text_batch, label_batch, length_batch = next(iter(dataloader))
print(text_batch)
print(label_batch)
print(length_batch)
print(text_batch.shape)
# 샘플들 중 가장 큰 크기가 사용됨

tensor([[   35,  1739,     7,   449,   721,     6,   301,     4,   787,     9,
             4,    18,    44,     1,  1705,  2460,   186,    25,     7,    24,
           100,  1874,  1739,    25,     7, 34415,  3568,  1103,  7517,   787,
             5,     1,  4991, 12401,    36,     7,   148,   111,   939,     6,
         11598,     1,   172,   135,    62,    25,  3199,  1602,     3,   928,
          1500,     9,     6,  4601,     1,   155,    36,    14,   274,     4,
         42945,     9,  4991,     3,    14, 10296,    34,  3568,     8,    51,
           148,    30,     1,    58,    16,    11,  1893,   125,     6,   420,
          1214,    27, 14542,   940,    11,     7,    29,   951,    18,    17,
         15994,   459,    34,  2480, 15211,  3713,     1,   840,  3200,     9,
          3568,    13,   107,     9,   175,    94,    25,    51, 10297,  1796,
            27,   712,    16,     1,   220,    17,     4,    54,   722,   238,
           395,     1,   787,    32,    27,  5236,  

In [38]:
# 배치 크기가 32인 데이터 로더를 만들어보자
batch_size = 32

train_dl = DataLoader(train_dataset, batch_size = batch_size,
                      shuffle=True, collate_fn=collate_batch)

valid_dl = DataLoader(valid_dataset, batch_size = batch_size,
                      shuffle=False, collate_fn=collate_batch)

test_dl = DataLoader(test_dataset, batch_size = batch_size,
                      shuffle=False, collate_fn=collate_batch)

### 문장 인코딩을 위한 임베딩 층
**원-핫 인코딩**
* 단어 집합의 크기만큼 벡터 차원을 가지고, 해당되는 단어가 있으면 1, 그렇지 않으면 0으로 표현하는 방식

* 어휘 사전의 크기가 커지면 차원의 저주로 인한 영향을 받음
* 또한 한개의 element를 제외하고 모든 원소가 0이므로 특성 벡터가 매우 희소해짐.

=> **임베딩**

In [39]:
#  nn.Embedding을 사용하여 임베딩 층을 간단히 만들 수 있음
embedding = nn.Embedding(
    num_embeddings=10,
    embedding_dim=3,
    padding_idx=0
)

# 네 개의 인덱스를 가진 샘플 두개로 구성된 배치
text_encoded_input = torch.LongTensor([[1,2,4,5],[4,3,2,0]])
print(embedding(text_encoded_input))

tensor([[[-0.6818, -0.5912,  0.2738],
         [-0.9649, -0.2358, -0.5710],
         [ 1.3949,  0.8082,  0.1530],
         [-0.4757, -1.8821, -0.7765]],

        [[ 1.3949,  0.8082,  0.1530],
         [ 0.0135, -0.5495, -1.0113],
         [-0.9649, -0.2358, -0.5710],
         [ 0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward0>)


### RNN 모델 만들기


In [40]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size):
    super().__init__()
    self.rnn = nn.RNN(input_size, hidden_size, num_layers=2,
                      batch_first=True)
    # self.rnn = nn.GRU(input_size, hidden_size, num_layers=2,
                      # batch_first= True)
    # self.rnn = nn.LSTM(input_size, hidden_size, num_layers=2,
                      #  batch_first=True)
    self.fc = nn.Linear(hidden_size, 1)

  def forward(self,x):
    _, hidden = self.rnn(x)
    out = hidden[-1, :, :] # hidden shape : [num_layers, batch_size, hidden_size]
                           # 마지막 은닉층의 최종 은닉 상태( at time seq_len-1)를 완전 연결 층의 입력으로 사용

    out = self.fc(out) # out shape before : [batch_size, hidden_size], out shape after : [batch_size,1]
    return out
model = RNN(64,32)
print(model)
model(torch.randn(5,3,64)) # batch_size = 5, seq_len = 3, input_size = 64 (input feature)

RNN(
  (rnn): RNN(64, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)


tensor([[-0.2167],
        [-0.5359],
        [-0.2935],
        [-0.1266],
        [-0.2043]], grad_fn=<AddmmBackward0>)

### 감성 분석 작업을 위한 RNN 모델 만들기
시퀀스가 길기 때문에 넓은 범위의 영향을 감지하기 위해 LSTM층을 사용해보자.  
먼저 크기가 20(embedding_dim=20)인 단어 임베딩을 만드는 임베딩 층으로 시작하여 감성 분석을 위한 RNN 모델을 만든후, LSTM 순환 층을 추가해보자.  마지막으로 은닉층으로 완전 연결 층을 추가하고 출력층으로 또 다른 완전 연결 층을 추가하자.  
출력층에는 로지스틱 시그모이드 활성화 함수를 통해 클래스 소속 확률 값 하나를 예측으로 반환하자.

In [41]:
class RNN(nn.Module):
  def __init__(self, vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size):
    super().__init__()
    self.embedding = nn.Embedding(
                             num_embeddings = vocab_size,
                             embedding_dim = embed_dim,
                             padding_idx = 0)
    self.rnn = nn.LSTM(embed_dim, rnn_hidden_size,
                       batch_first = True)
    self.fc1 = nn.Linear(rnn_hidden_size, fc_hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(fc_hidden_size, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, text, lengths):
    out = self.embedding(text)
    out = nn.utils.rnn.pack_padded_sequence(
        out, lengths.cpu().numpy(), enforce_sorted = False, batch_first = True
    )
    output, (h_n, c_n) = self.rnn(out)
    out = h_n[-1,:,:]
    out = self.fc1(out)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    return out

vocab_size = len(vocab)
embed_dim = 20
rnn_hidden_size = 64
fc_hidden_size = 64
model = RNN(vocab_size, embed_dim,
            rnn_hidden_size, fc_hidden_size)
model

RNN(
  (embedding): Embedding(69025, 20, padding_idx=0)
  (rnn): LSTM(20, 64, batch_first=True)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [42]:
# 한 에포크 동안 주어진 데이터셋에서 모델을 훈련하고 분류 정확도와 손실을 반환하는 train 함수를 만들자
import numpy as np
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
def train(dataloader):
  model.train()
  total_acc, total_loss = 0, 0
  for text_batch, label_batch, lengths in dataloader:
    optimizer.zero_grad()
    pred = model(text_batch, lengths)[:,0] # converts into a tensor of shape (batch_size, )
    loss = loss_fn(pred, label_batch)
    total_acc += ( (pred>=0.5).float() == label_batch).float().sum().item() #.item() extracts scalar value from tensor
    total_loss += loss.item() * label_batch.size(0) # label_batch.size(0) represents the batch size
  return total_acc/len(dataloader.dataset), \
         total_loss/len(dataloader.dataset)

In [43]:
# 비슷하게 주어진 데이터셋에서 모델의 성능을 평가하는 evaluate 함수를 만들자
def evaluate(dataloader):
  model.eval()
  total_acc, total_loss = 0, 0
  with torch.no_grad():
    for text_batch, label_batch, lengths in dataloader:
      pred = model(text_batch, lengths)[:,0] # converts into a tensor of shape (batch_size, ) from (batch_size, 1)
      loss = loss_fn(pred, label_batch)
      total_acc += (
          (pred >= 0.5).float() == label_batch
      ).float().sum().item()
      total_loss += loss.item() * label_batch.size(0)
  return total_acc/len(list(dataloader.dataset)), \
         total_loss/len(list(dataloader.dataset))

In [ ]:
# 열 번의 에포크 동안 모델을 훈련하고 훈련 및 검증 성능을 표시해보자
num_epochs = 10
torch.manual_seed(1)
for epoch in range(num_epochs):
  acc_train, loss_train = train(train_dl)
  acc_valid, loss_valid = evaluate(valid_dl)
  print(f'에포크 {epoch} 정확도 : {acc_train:.4f}'
        f' 검증 정확도 : {acc_valid:.4f}')

In [ ]:
# 훈련이 끝난 후 테스트 데이터로 평가해보자
acc_test, loss_test = evaluate(test_dl)
print(f'테스트 정확도 : {acc_test}')

### 양방향 RNN


In [49]:
class RNN(nn.Module):
  def __init__(self, vocab_size, embed_dim, rnn_hidden_size,
               fc_hidden_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embed_dim,
                                padding_idx = 0)
    self.rnn = nn.LSTM(embed_dim, rnn_hidden_size,
                     num_layers = 1, batch_first = True,
                     bidirectional = True)
    self.fc1 = nn.Linear(2*rnn_hidden_size, fc_hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(fc_hidden_size, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, text, lengths):
    out = self.embedding(text)
    out = nn.utils.rnn.pack_padded_sequence(
        out, lengths.cpu().numpy(), batch_first = True, enforce_sorted=False
    )
    output, (h_n, c_n) = self.rnn(out)
    out = torch.cat(
        (h_n[-2, :, :],
         h_n[-1, :, :]), dim = 1)
    out = self.fc1(out)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    return out

loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
def train(dataloader):
  model.train()
  total_acc, total_loss = 0, 0
  for text_batch, label_batch, lengths in dataloader:
    optimizer.zero_grad()
    pred = model(text_batch, lengths)[:,0] # converts into a tensor of shape (batch_size, )
    loss = loss_fn(pred, label_batch)
    total_acc += ( (pred>=0.5).float() == label_batch).float().sum().item() #.item() extracts scalar value from tensor
    total_loss += loss.item() * label_batch.size(0) # label_batch.size(0) represents the batch size
  return total_acc/len(dataloader.dataset), \
         total_loss/len(dataloader.dataset)

def evaluate(dataloader):
  model.eval()
  total_acc, total_loss = 0, 0
  with torch.no_grad():
    for text_batch, label_batch, lengths in dataloader:
      pred = model(text_batch, lengths)[:,0] # converts into a tensor of shape (batch_size, ) from (batch_size, 1)
      loss = loss_fn(pred, label_batch)
      total_acc += (
          (pred >= 0.5).float() == label_batch
      ).float().sum().item()
      total_loss += loss.item() * label_batch.size(0)
  return total_acc/len(list(dataloader.dataset)), \
         total_loss/len(list(dataloader.dataset))

In [50]:
torch.manual_seed(1)
model = RNN(vocab_size, embed_dim,
            rnn_hidden_size, fc_hidden_size)
model

RNN(
  (embedding): Embedding(69025, 20, padding_idx=0)
  (rnn): LSTM(20, 64, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [ ]:

num_epochs = 10
torch.manual_seed(1)
for epoch in range(num_epochs):
  acc_train, loss_train = train(train_dl)
  acc_valid, loss_valid = evaluate(valid_dl)
  print(f'에포크 {epoch} 정확도 : {acc_train:.4f}'
        f' 검증 정확도 : {acc_valid:.4f}')

In [ ]:
acc_test, loss_test = evaluate(test_dl)
print(f'테스트 정확도 : {acc_test}')